In [ ]:
import glob
import os
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

# 1. 폴더 경로 지정 (구글드라이브나 로컬 환경에 맞게 수정)
file_path = '/content/drive/MyDrive/보건의료데이터 공모전/병합_최종.csv'
df= pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,age,BP1,BP5,BP6_10,BP6_2,BP6_31,BP7,식품코드,식품군 분류,...,가구소득분위,음주빈도,주간근력운동일수,HE_BMI,흡연여부,비만여부,비만단계,주요경로_당_섭취량,주요경로_비율(%),나이
0,YA84242703,12.0,3.0,2.0,2.0,2.0,2.0,2.0,3004.0,당류,...,5.0,1.0,1.0,17.641364,비흡연,비비만,저체중,53.634673,1.172715,12.0
1,YA84384302,12.0,2.0,2.0,2.0,2.0,2.0,1.0,3014.0,당류,...,3.0,1.0,1.0,24.508946,비흡연,비비만,과체중,17.682180,2.586764,12.0
2,YA84384303,15.0,3.0,2.0,2.0,2.0,2.0,2.0,8036.0,과일류,...,3.0,1.0,3.0,27.837219,비흡연,비만,1단계 비만,15.511864,9.659593,15.0
3,YA84390404,34.0,3.0,2.0,2.0,2.0,2.0,2.0,3014.0,당류,...,5.0,4.0,1.0,25.807581,비흡연,비만,1단계 비만,11.058057,0.762873,34.0
4,YA84412603,13.0,3.0,2.0,2.0,2.0,2.0,2.0,13035.0,우유류,...,3.0,1.0,6.0,20.931313,비흡연,비비만,정상체중,96.962383,8.014211,13.0


In [ ]:
b = ['성별', '가구소득분위', '음주빈도', '주간근력운동일수', '흡연여부', '비만여부', '비만단계']

나이 나눠서

In [ ]:
# 전체 데이터프레임이 df라고 가정

# 청소년 (12세 ~ 19세)
df_teen = df[(df['나이'] >= 12) & (df['나이'] <= 19)].copy()
df_teen = df_teen.drop(columns=['음주빈도', '흡연여부'], errors='ignore')

# 청년 (20세 ~ 34세)
df_young = df[(df['나이'] >= 20) & (df['나이'] <= 34)].copy()

In [ ]:
df_teen.columns

Index(['ID', 'age', 'BP1', 'BP5', 'BP6_10', 'BP6_2', 'BP6_31', 'BP7', '식품코드',
       '식품군 분류', '식품별 섭취량', '3차 식품코드별 섭취량', '에너지 섭취량', '당 섭취량', '식품코드명',
       '나이_x', '총 당 섭취량(g)', '총 에너지 섭취량(kcal)', '당섭취열량(kcal)', '당섭취비율_g(%)',
       '당섭취비율_kcal(%)', '당 에너지 비율 WHO(%)', '보정 여부', '분류', 'mental_composite',
       '성별', '나이_y', '가구소득분위', '주간근력운동일수', 'HE_BMI', '비만여부', '비만단계',
       '주요경로_당_섭취량', '주요경로_비율(%)', '나이', '비율구간'],
      dtype='object')

In [ ]:
mental = ['BP1', 'BP5', 'BP6_10', 'BP6_2', 'BP6_31', 'BP7', 'mental_composite']

In [ ]:
for col in covariates:
    print(f"{col}: {df[col].nunique()}개 → {df[col].unique()}")


성별: 2개 → [2. 1.]
가구소득분위: 5개 → [ 5.  3.  2.  4.  1. nan]
음주빈도: 7개 → [1. 4. 8. 2. 3. 5. 6.]
주간근력운동일수: 7개 → [1. 3. 6. 4. 2. 5. 9.]
흡연여부: 2개 → ['비흡연' '흡연']
비만여부: 2개 → ['비비만' '비만']
비만단계: 5개 → ['저체중' '과체중' '1단계 비만' '정상체중' '2단계 이상 비만']


변수에 따른 p-value 계산

In [ ]:
from scipy.stats import chi2_contingency, mannwhitneyu, spearmanr, pearsonr
import numpy as np
import pandas as pd

def get_covariates(df):
    binary_covariates = ['성별', '비만여부'] # 명목형 이진
    ordinal_covariates = ['가구소득분위', '주간근력운동일수', '비만단계'] # 순서형

    # '흡연여부'가 열에 있으면 포함
    if '흡연여부' in df.columns:
        binary_covariates.append('흡연여부') # 명목형 이진

    # '음주빈도'가 열에 있으면 포함
    if '음주빈도' in df.columns:
        ordinal_covariates.append('음주빈도') # 순서형

    return binary_covariates, ordinal_covariates

# 정신건강 변수 분류
mental_binary = ['BP5', 'BP6_10', 'BP6_2', 'BP6_31', 'BP7']  # 이진형
mental_ordinal = ['BP1']  # 순서형
mental_continuous = ['mental_composite']  # 연속형


In [ ]:
def analyze_mental_relationship(df, group_name):
    results = []
    binary_covariates, ordinal_covariates = get_covariates(df)
    all_covariates = binary_covariates + ordinal_covariates

    if '흡연여부' in df.columns:
        smoking_map = {
        '비흡연': 0,
        '흡연': 1
         }
        df['흡연여부'] = df['흡연여부'].map(smoking_map)


    obese_map = {
        '비비만': 0,
        '비만': 1
        }
    df['비만여부'] = df['비만여부'].map(obese_map)
    # 비만단계 매핑 (저체중 → 2단계 이상 비만)
    bmi_stage_map = {
    '저체중': 1,
    '정상체중': 2,
    '과체중': 3,
    '1단계 비만': 4,
    '2단계 이상 비만': 5
    }

    df['비만단계'] = df['비만단계'].map(bmi_stage_map)


    for cov in all_covariates:
        for mental in mental_binary + mental_ordinal + mental_continuous:
            sub_df = df[[cov, mental]].dropna()
            try:
                if mental in mental_binary:
                    if cov in binary_covariates:
                        ct = pd.crosstab(sub_df[cov], sub_df[mental])
                        _, p = chi2_contingency(ct)[:2]
                        method = 'Chi-square'
                    elif cov in ordinal_covariates:
                        group0 = sub_df[sub_df[mental] == 0][cov]
                        group1 = sub_df[sub_df[mental] == 1][cov]
                        _, p = mannwhitneyu(group0, group1)
                        method = 'Mann–Whitney U'

                elif mental in mental_ordinal:
                    if cov in binary_covariates:
                        u1 = sub_df[sub_df[cov] == sub_df[cov].unique()[0]][mental]
                        u2 = sub_df[sub_df[cov] == sub_df[cov].unique()[1]][mental]
                        _, p = mannwhitneyu(u1, u2)
                        method = 'Mann–Whitney U'
                    elif cov in ordinal_covariates:
                        _, p = spearmanr(sub_df[cov], sub_df[mental])
                        method = 'Spearman'

                elif mental in mental_continuous:
                    if cov in binary_covariates:
                        g1 = sub_df[sub_df[cov] == sub_df[cov].unique()[0]][mental]
                        g2 = sub_df[sub_df[cov] == sub_df[cov].unique()[1]][mental]
                        _, p = mannwhitneyu(g1, g2)
                        method = 'Mann–Whitney U'
                    elif cov in ordinal_covariates:
                        _, p = spearmanr(sub_df[cov], sub_df[mental])
                        method = 'Spearman'

                results.append({
                    '연령대': group_name,
                    '공변량': cov,
                    '정신건강변수': mental,
                    '검정방법': method,
                    'p-value': round(p, 4),
                    '유의': '✅' if p < 0.05 else '❌'
                })

            except:
                results.append({
                    '연령대': group_name,
                    '공변량': cov,
                    '정신건강변수': mental,
                    '검정방법': 'Error',
                    'p-value': 'NaN',
                    '유의': '❌'
                })

    return pd.DataFrame(results)


In [ ]:
# 실행
df_teen_result = analyze_mental_relationship(df_teen, '청소년(12~19세)')
df_young_result = analyze_mental_relationship(df_young, '청년(20~34세)')

# 병합
df_all_result = pd.concat([df_teen_result, df_young_result], ignore_index=True)

# 유의한 것만 보기
df_significant = df_all_result[df_all_result['유의'] == '✅']

# 결과 출력
import pandas as pd
pd.set_option('display.max_rows', None)
display(df_significant)


/tmp/ipython-input-447832718.py:43: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p = mannwhitneyu(group0, group1)
/tmp/ipython-input-447832718.py:43: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p = mannwhitneyu(group0, group1)
/tmp/ipython-input-447832718.py:43: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p = mannwhitneyu(group0, group1)
/tmp/ipython-input-447832718.py:43: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  _, p = mannwhitneyu(group0, group1)
/tmp/ipython-input-447832718.py:43: SmallSampleWarning: One or more sample arguments is too small; all returned values will be N

,연령대,공변량,정신건강변수,검정방법,p-value,유의
0,청소년(12~19세),성별,BP5,Chi-square,0.0021,✅
1,청소년(12~19세),성별,BP6_10,Chi-square,0.0485,✅
4,청소년(12~19세),성별,BP7,Chi-square,0.0188,✅
5,청소년(12~19세),성별,BP1,Mann–Whitney U,0.0002,✅
6,청소년(12~19세),성별,mental_composite,Mann–Whitney U,0.0000,✅
12,청소년(12~19세),비만여부,BP1,Mann–Whitney U,0.0001,✅
13,청소년(12~19세),비만여부,mental_composite,Mann–Whitney U,0.0002,✅
33,청소년(12~19세),비만단계,BP1,Spearman,0.0002,✅
34,청소년(12~19세),비만단계,mental_composite,Spearman,0.0002,✅
35,청년(20~34세),성별,BP5,Chi-square,0.0000,✅


In [ ]:
from scipy.stats import mannwhitneyu, spearmanr
import pandas as pd

# 주요경로_비율(%) 변수명
independent = '주요경로_비율(%)'

# 공변량 정의 함수
def get_covariates(df):
    binary_covariates = ['성별', '비만여부']
    ordinal_covariates = ['가구소득분위', '주간근력운동일수', '비만단계']

    if '흡연여부' in df.columns:
        binary_covariates.append('흡연여부')
    if '음주빈도' in df.columns:
        ordinal_covariates.append('음주빈도')

    return binary_covariates, ordinal_covariates

# 분석 함수
def analyze_path_ratio_relationship(df, group_name):
    binary_covariates, ordinal_covariates = get_covariates(df)
    results = []

    for cov in binary_covariates:
        try:
            sub_df = df[[cov, independent]].dropna()
            groups = sub_df[cov].unique()
            if len(groups) == 2:
                g1 = sub_df[sub_df[cov] == groups[0]][independent]
                g2 = sub_df[sub_df[cov] == groups[1]][independent]
                _, p = mannwhitneyu(g1, g2)
                method = 'Mann–Whitney U'

                results.append({
                    '연령대': group_name,
                    '공변량': cov,
                    '검정방법': method,
                    'p-value': round(p, 4),
                    '유의': '✅' if p < 0.05 else '❌'
                })
        except:
            results.append({
                '연령대': group_name,
                '공변량': cov,
                '검정방법': 'Error',
                'p-value': 'NaN',
                '유의': '❌'
            })

    for cov in ordinal_covariates:
        try:
            sub_df = df[[cov, independent]].dropna()
            _, p = spearmanr(sub_df[cov], sub_df[independent])
            method = 'Spearman'

            results.append({
                '연령대': group_name,
                '공변량': cov,
                '검정방법': method,
                'p-value': round(p, 4),
                '유의': '✅' if p < 0.05 else '❌'
            })
        except:
            results.append({
                '연령대': group_name,
                '공변량': cov,
                '검정방법': 'Error',
                'p-value': 'NaN',
                '유의': '❌'
            })

    return pd.DataFrame(results)


In [ ]:
# 각 연령대별 분석 실행
df_teen_result = analyze_path_ratio_relationship(df_teen, '청소년(12~19세)')
df_young_result = analyze_path_ratio_relationship(df_young, '청년(20~34세)')

# 병합 및 유의한 결과 필터링
df_all_result = pd.concat([df_teen_result, df_young_result], ignore_index=True)
df_significant = df_all_result[df_all_result['유의'] == '✅']

# 결과 출력
pd.set_option('display.max_rows', None)
display(df_significant)


,연령대,공변량,검정방법,p-value,유의
0,청소년(12~19세),성별,Mann–Whitney U,0.0221,✅
7,청년(20~34세),흡연여부,Mann–Whitney U,0.0046,✅
11,청년(20~34세),음주빈도,Spearman,0.0227,✅


✅ 1. 주요당경로_비율과 공변량 간 유의한 관계

🔎 요약:
→ 성별, 흡연여부, 음주빈도는 주요경로당섭취비율(%)과 유의한 관련이 있음 → 향후 분석 시 보정변수로 고려 가능

✅ 2. 정신건강 변수와 공변량 간 유의한 관계
연령대: 청소년(12~19세)

성별: 자살 생각(BP6_10), 스트레스(BP1), 정신건강 지수(mental_composite) 등과 유의미한 차이 있음

비만여부 / 비만단계: 스트레스 인지 수준과 유의미한 관계 존재

스트레스 인지율(BP1): 주요 변수들과 Spearman 상관, Mann–Whitney U 결과 유의

연령대: 청년(20~34세)

성별: 거의 모든 정신건강 변수와 유의미한 차이 (BP5, BP6_10, BP7 등)

흡연여부: 자살계획(BP6_2), 스트레스(BP1), 정신건강지수 등과 유의

주간근력운동일수 / 가구소득분위: 스트레스, 정신건강 지수와 상관관계 있음

비만여부 / 비만단계: 스트레스 지수 및 mental_composite와도 관계 존재

🔎 요약:
→ 성별, 흡연여부, 비만 관련 변수, 소득/운동 등 생활 습관 변수들이 정신건강과 유의하게 연결
→ 정신건강과의 교차분석 시 주요 공변량으로 충분히 고려할 수 있음

🔚 전체 정리
공변량들이 주요당섭취비율 및 정신건강지표와도 연결되어 있음

이 결과는 이후 중다회귀분석이나 공변량 보정(Covariate Adjustment) 시 변수 선택에 근거로 사용 가능

특히, 성별, 흡연여부, 비만여부, 스트레스 인지 수준, 음주빈도 등은 반복적으로 유의하게 나타남